This notebook contains examples of some typical queries used to find statistical leaders and trends in USports women's basketball since the 2009-2010 season
Results usually output in a dataframe 

In [1]:
import pandas as pd
from sql_functions import create_db_connection, execute_query, read_query

In [2]:
#connect to database to start
pw = "Queens2021!"
connection = create_db_connection("localhost", "root", pw, "usports")

MySQL Database connection successful


In [3]:
#get highest average for a stat for each season for players playing more than 10 games
q = """
    SELECT max(ppg), player_shooting.season
    FROM players 
    JOIN player_shooting 
    ON players.player_id = player_shooting.player_id
    JOIN player_info
    ON player_shooting.player_id = player_info.player_id AND
       player_shooting.season = player_info.season
    WHERE games_played > 10
    GROUP BY player_shooting.season
    ORDER BY player_shooting.season
    """
result = read_query(connection, q, None)
result
df = pd.DataFrame(result)
df

,0,1
0,22.1,2010
1,25.8,2011
2,24.4,2012
3,28.3,2013
4,23.6,2014
5,28.8,2015
6,20.8,2016
7,22.8,2017
8,25.9,2018
9,26.4,2019


In [4]:
# find the player who recorded highest average for a stat for each season (who played more than 10 games)
g_played = 10
for t in result:
    t_ppg = t[0]-0.1
    t_season = t[1]
    vars = [t_ppg, t_season, g_played]

    q = """
        SELECT name, ppg, player_shooting.season
        FROM players 
        JOIN player_shooting 
        ON players.player_id = player_shooting.player_id
        JOIN player_info
        ON player_shooting.player_id = player_info.player_id AND
           player_shooting.season = player_info.season
        WHERE ppg > %s AND player_shooting.season = %s AND games_played > %s
        ORDER BY player_shooting.season;
        """
    query_result = read_query(connection, q, vars)
    print(query_result)

[('Kelsey Hodgson', 22.1, 2010)]
[('Justine Colley', 25.8, 2011)]
[('Justine Colley', 24.4, 2012)]
[('Justine Colley', 28.3, 2013)]
[('Cassie Cooke', 23.6, 2014)]
[('Jylisa Williams', 28.8, 2015)]
[('Lindsay Lessard', 20.8, 2016)]
[('Paloma Anderson', 22.8, 2017)]
[('Antoinette Miller', 25.9, 2018)]
[('Hannah Brown', 26.4, 2019)]
[('Haille Nickerson', 24.6, 2020)]
[('Jayda Veinot', 23.4, 2022)]


In [5]:
#get player id of players who recorded certain statistical milestones (as well as the statistics themselves)
q = """
    SELECT player_shooting.player_id, ppg, reb
    FROM player_shooting 
    JOIN player_ballcontrol
    ON player_shooting.player_id = player_ballcontrol.player_id 
    AND player_shooting.season = player_ballcontrol.season
    JOIN player_info
    ON player_shooting.player_id = player_info.player_id
    AND player_shooting.season = player_info.season
    WHERE ppg > 20 AND reb > 10 AND games_played > 10;
    """
query_result = read_query(connection, q, None)
df = pd.DataFrame(query_result, columns=['Player ID', 'PPG', 'Rebounds'])
df

,Player ID,PPG,Rebounds
0,1395,20.2,10.8
1,1566,26.4,12.6
2,2055,23.8,10.3
3,2376,22.3,14.1
4,2376,20.6,12.5
5,2994,22.7,15.7


In [6]:
#get name and player id of those who recorded certain statistical milestones in a season
q = """
    SELECT name, player_id
    FROM players
    WHERE player_id IN (SELECT player_shooting.player_id
                        FROM player_shooting 
                        JOIN player_ballcontrol
                        ON player_shooting.player_id = player_ballcontrol.player_id 
                        AND player_shooting.season = player_ballcontrol.season
                        JOIN player_info
                        ON player_shooting.player_id = player_info.player_id
                        AND player_shooting.season = player_info.season
                        WHERE ppg > 20 AND reb > 10 AND games_played > 10);
    """
    
query_result = read_query(connection, q, None)
df = pd.DataFrame(query_result)
df

,0,1
0,Jylisa Williams,1395
1,Hannah Brown,1566
2,Faith Hezekiah,2055
3,Leashja Grant,2376
4,Kiyara Letlow,2994


In [7]:
#get name, player id, season, and team of players who recorded certain statistical milestones
# - an alternative to the method above to return more information and players who achieved milestone multiple times
q = """
    SELECT name, player_id, season, team
    FROM players 
    NATURAL JOIN player_shooting 
    NATURAL JOIN player_ballcontrol
    NATURAL JOIN player_info
    WHERE ppg > 20 AND reb > 10 AND games_played > 10;
    """                  

query_result = read_query(connection, q, None)
df = pd.DataFrame(query_result)
df

,0,1,2,3
0,Jylisa Williams,1395,2014,lakehead
1,Hannah Brown,1566,2019,capebreton
2,Faith Hezekiah,2055,2019,winnipeg
3,Leashja Grant,2376,2018,lakehead
4,Leashja Grant,2376,2019,lakehead
5,Kiyara Letlow,2994,2022,capebreton


In [8]:
#Count number of seasons each player achieved the statistical milestones
q = """
    SELECT name, count(name)
    FROM players 
    NATURAL JOIN player_shooting 
    NATURAL JOIN player_ballcontrol
    NATURAL JOIN player_info
    WHERE ppg > 20 AND reb > 10 AND games_played > 10
    GROUP BY name;
    """                  

query_result = read_query(connection, q, None)
df = pd.DataFrame(query_result)
df

,0,1
0,Jylisa Williams,1
1,Hannah Brown,1
2,Faith Hezekiah,1
3,Leashja Grant,2
4,Kiyara Letlow,1


In [9]:
#get highest average for a stat for each jersey number for players playing more than 10 games
q = """
    SELECT max(ppg), number
    FROM players 
    NATURAL JOIN player_shooting 
    NATURAL JOIN player_info
    WHERE games_played > 10
    GROUP BY number
    ORDER BY number;
    """
result = read_query(connection, q, None)
result
df = pd.DataFrame(result)
df

,0,1
0,7.5,NaN
1,9.0,0.0
2,25.9,1.0
3,24.2,2.0
4,20.1,3.0
5,28.8,4.0
6,28.3,5.0
7,19.3,6.0
8,19.7,7.0
9,21.6,8.0


In [10]:
# find the player who recorded highest average for each jersey number (who played more than 10 games)
g_played = 10
idx = 0
for t in result:
    if idx == 0: #skip None Number
        idx += 1
        continue
    t_stat = t[0]-0.1
    t_group = t[1]
    vars = [t_stat, t_group, g_played]

    q = """
        SELECT name, ppg, player_info.number
        FROM players 
        NATURAL JOIN player_shooting 
        NATURAL JOIN player_info
        WHERE ppg > %s AND player_info.number = %s AND games_played > %s;
        """
    query_result = read_query(connection, q, vars)
    print(query_result)

[('Jada Poon-TIp', 9.0, 0)]
[('Antoinette Miller', 25.9, 1)]
[('Haley McDonald', 24.2, 2)]
[('Amira Giannattasio', 20.1, 3)]
[('Jylisa Williams', 28.8, 4)]
[('Justine Colley', 28.3, 5)]
[('Korissa Williams', 19.3, 6)]
[('Ashlyn Day', 19.7, 7)]
[('Sarah Gates', 21.6, 8)]
[('Faith Hezekiah', 23.8, 9)]
[('Taylor Claggett', 19.6, 10)]
[('Paloma Anderson', 22.8, 11)]
[('Amanda Sharpe', 20.4, 12)]
[('Dalyce Emmerson', 19.2, 13)]
[('Cassie Cooke', 23.6, 14)]
[('Cassie Cooke', 23.6, 15)]
[('Abby Smith', 4.4, 16)]
[('Sarah Harvey', 12.7, 17)]
[('Kyla Smith', 7.6, 18)]
[('Whitney Ellenor', 15.9, 20)]
[('Hannah Sunley-Paisley', 18.9, 21)]
[('Kira Cornelissen', 20.0, 22)]
[('Haille Nickerson', 24.6, 23)]
[('Brooklyn Wright', 14.9, 24)]
[('Laura Meadows', 11.0, 25)]
[('Stephanie Marin', 2.0, 30)]
[('Grace Fishbein', 16.7, 31)]
[('Megan Looney', 13.4, 32)]
[('Tianna Brown', 11.3, 33)]
[('Hannah Brown', 26.4, 34)]
[('Taylor McAllister', 2.7, 44), ('Morgan Tamminga', 2.8, 44)]
[('Kendra Cole', 4.4, 50

In [11]:
# count number of players who took greater than X three pointers in each season
q = """
    SELECT season, count(name)
    FROM players
    NATURAL JOIN player_shooting
    WHERE fga3 > 6
    GROUP BY season;
    """

result = read_query(connection, q, None)
result
df = pd.DataFrame(result)
df

,0,1
0,2013,13
1,2010,17
2,2015,13
3,2016,18
4,2012,7
5,2011,8
6,2014,10
7,2017,13
8,2018,18
9,2019,24


In [12]:
# count number of players who averaged over 35% 3FG and took at least 2 per game in each season
q = """
    SELECT season, count(name)
    FROM players
    NATURAL JOIN player_shooting
    WHERE fga3 > 2 AND fg3_percent > 35
    GROUP BY season;
    """

result = read_query(connection, q, None)
result
df = pd.DataFrame(result)
df

,0,1
0,2010,42
1,2012,48
2,2011,42
3,2015,36
4,2013,35
5,2014,28
6,2016,35
7,2017,33
8,2018,42
9,2019,45
